<a href="https://colab.research.google.com/github/nonyeezeh/Research-Project-Code/blob/main/LBN_Dense_BIC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [40]:
pip install pgmpy

In [202]:
import numpy as np
import pandas as pd
from pgmpy.estimators import MmhcEstimator, BicScore, MaximumLikelihoodEstimator
from pgmpy.models import BayesianNetwork
from sklearn.model_selection import train_test_split
from scipy.stats import entropy

# Bayesian Network Data Generation 1000, 2000, ..., 10000 Samples (dense)

In [204]:
# Function to generate CPDs
def generate_cpds():
    # Generate random probabilities for IR
    ir_probs = np.random.rand(3)
    ir_probs /= ir_probs.sum()  # Normalize to make it a valid probability distribution

    # Generate random probabilities for EI given IR
    ei_given_ir_probs = np.random.rand(3, 3)
    ei_given_ir_probs /= ei_given_ir_probs.sum(axis=0, keepdims=True)

    # Generate random probabilities for SP given IR and EI
    sp_probs = np.random.rand(3, 3, 3)
    sp_probs /= sp_probs.sum(axis=0, keepdims=True)

    return ir_probs, ei_given_ir_probs, sp_probs

# Function to generate and save samples
def generate_and_save_samples(ir_probs, ei_probs, sp_probs, sample_size, filename):
    output_data = []

    # Generate `sample_size` random samples
    for _ in range(sample_size):
        # Sample `IR` state based on `IR` probabilities
        ir_state_idx = np.random.choice(3, p=ir_probs)
        ir_state = ['low', 'medium', 'high'][ir_state_idx]
        ir_prob = ir_probs[ir_state_idx]

        # Sample `EI` state based on `EI` probabilities given `IR`
        ei_probs_given_ir = ei_probs[:, ir_state_idx]
        ei_state_idx = np.random.choice(3, p=ei_probs_given_ir)
        ei_state = ['poor', 'average', 'good'][ei_state_idx]
        ei_prob = ei_probs_given_ir[ei_state_idx]

        # Sample `SP` state based on `SP` probabilities given `IR` and `EI`
        sp_probs_given_ir_ei = sp_probs[:, ir_state_idx, ei_state_idx]
        sp_state_idx = np.random.choice(3, p=sp_probs_given_ir_ei)
        sp_state = ['decrease', 'stable', 'increase'][sp_state_idx]
        sp_prob = sp_probs_given_ir_ei[sp_state_idx]

        # Append sample data to output list
        output_data.append({
            'IR_State': ir_state,
            'IR_Prob': f'{ir_prob:.4f}',
            'EI_State': ei_state,
            'EI_Prob': f'{ei_prob:.4f}',
            'SP_Probabilities (decrease, stable, increase)': ', '.join([f'{prob:.4f}' for prob in sp_probs_given_ir_ei]),
            'Chosen_SP_State': sp_state,
            'Chosen_SP_Probability': f'{sp_prob:.4f}'
        })

    # Create a DataFrame from the output data
    output_df = pd.DataFrame(output_data)

    # Save the output DataFrame to a CSV file
    output_df.to_csv(filename, index=False)

    # Print the first few rows for visual confirmation
    print(f"\nSample size: {sample_size} - First few rows of generated samples:\n")
    print(tabulate(output_df.head(), headers='keys', tablefmt='grid'))

# Generate and save samples for sample sizes from 1000 to 10000 every 1000
sample_sizes = range(1000, 11000, 1000)

for size in sample_sizes:
    # Generate the CPDs
    ir_probs, ei_given_ir_probs, sp_probs = generate_cpds()

    # Generate and save individual samples for the given sample size
    generate_and_save_samples(ir_probs, ei_given_ir_probs, sp_probs, size, f'combined_probabilities_{size}.csv')

# Notify the user that the process is done
print("\nGeneration and saving of individual samples complete for all sample sizes!")


Sample size: 1000 - First few rows of generated samples:

+----+------------+-----------+------------+-----------+-------------------------------------------------+-------------------+-------------------------+
|    | IR_State   |   IR_Prob | EI_State   |   EI_Prob | SP_Probabilities (decrease, stable, increase)   | Chosen_SP_State   |   Chosen_SP_Probability |
+====+============+===========+============+===========+=================================================+===================+=========================+
|  0 | medium     |    0.8201 | good       |    0.4786 | 0.4008, 0.2449, 0.3543                          | decrease          |                  0.4008 |
+----+------------+-----------+------------+-----------+-------------------------------------------------+-------------------+-------------------------+
|  1 | low        |    0.1231 | good       |    0.5784 | 0.3598, 0.1661, 0.4741                          | increase          |                  0.4741 |
+----+------------+----

# LBN Dense BIC & Entropy

Max-Min Hill Climbing

In [205]:
# Sample sizes to loop through
sample_sizes = range(1000, 11000, 1000)

# Initialize list to store K-L divergence and standard deviation values for each sample size
results = []

# Small smoothing value to avoid zero probabilities
epsilon = 1e-10

# Loop through each sample size
for sample_size in sample_sizes:
    print(f"\nProcessing sample size: {sample_size}")

    # Load the dense dataset for the current sample size
    dense_data_file = f'combined_probabilities_{sample_size}.csv'
    df_dense = pd.read_csv(dense_data_file)

    # Manually encode categorical variables for IR, EI, and SP
    ir_map = {'low': 0, 'medium': 1, 'high': 2}
    ei_map = {'poor': 0, 'average': 1, 'good': 2}
    sp_map = {'decrease': 0, 'stable': 1, 'increase': 2}

    df_dense['IR_encoded'] = df_dense['IR_State'].map(ir_map)
    df_dense['EI_encoded'] = df_dense['EI_State'].map(ei_map)
    df_dense['SP_encoded'] = df_dense['Chosen_SP_State'].map(sp_map)

    # Split the data into training, validation, and test sets
    X = df_dense[['IR_encoded', 'EI_encoded']]
    y = df_dense['SP_encoded']

    # Split into training (70%) and temp (30%) for validation and test
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)
    # Split temp into validation (50%) and test (50%)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, shuffle=True)

    # Concatenate X and y to form the training set for learning the BN structure
    df_train = pd.concat([X_train, y_train], axis=1)

    # Define the Max-Min Hill Climbing (MMHC) structure learning algorithm
    mmhc = MmhcEstimator(df_train)
    scoring_method = BicScore(df_train)

    # Estimate the best structure using MMHC
    best_dag = mmhc.estimate(scoring_method=scoring_method)
    best_model = BayesianNetwork(best_dag.edges())

    # Display the learned structure (edges of the Bayesian Network)
    print(f"\nLearned Structure (Edges) for {sample_size} samples:")
    print(best_model.edges())

    # Learn the CPDs using Maximum Likelihood Estimation (MLE)
    best_model.fit(df_train, estimator=MaximumLikelihoodEstimator)

    # Check if the model is valid after learning the parameters
    assert best_model.check_model()

    # --- K-L Divergence and Standard Deviation Calculation Block ---
    # Calculate the probabilities using the learned BN and the test set
    inference = best_model.predict(X_test)
    predicted_probabilities = inference['SP_encoded']

    # Calculate K-L divergence and standard deviation between the ground truth and learned BN
    ground_truth_probabilities = y_test.value_counts(normalize=True).sort_index()
    predicted_probabilities = predicted_probabilities.value_counts(normalize=True).sort_index()

    # Reindex both distributions to have the same set of categories and add smoothing
    all_categories = sorted(set(ground_truth_probabilities.index).union(set(predicted_probabilities.index)))
    ground_truth_probabilities = ground_truth_probabilities.reindex(all_categories, fill_value=epsilon)
    predicted_probabilities = predicted_probabilities.reindex(all_categories, fill_value=epsilon)

    # Calculate K-L divergence with smoothing
    kl_divergence = entropy(pk=ground_truth_probabilities, qk=predicted_probabilities)

    # Standard deviation between predicted probabilities and actual probabilities
    std_dev = np.std(predicted_probabilities - ground_truth_probabilities)

    # Append results for this sample size
    results.append({
        'Sample_Size': sample_size,
        'K-L_Divergence': kl_divergence,
        'Standard_Deviation': std_dev
    })

    # Print the K-L divergence and standard deviation for this sample size
    print(f"\nResults for sample size {sample_size}:")
    print(f"K-L Divergence: {kl_divergence:.4f}")
    print(f"Standard Deviation: {std_dev:.4f}")

# After processing all sample sizes, save results to CSV and display them
results_df = pd.DataFrame(results)
results_df.to_csv('kl_std_results_mmhc.csv', index=False)

# Print all results after saving to CSV
print("\nAll results have been saved to 'kl_std_results_mmhc.csv'.")
print(results_df)


Processing sample size: 1000


  0%|          | 0/1000000 [00:00<?, ?it/s]


Learned Structure (Edges) for 1000 samples:
[('EI_encoded', 'IR_encoded'), ('SP_encoded', 'EI_encoded')]


  0%|          | 0/9 [00:00<?, ?it/s]


Results for sample size 1000:
K-L Divergence: 6.2300
Standard Deviation: 0.2816

Processing sample size: 2000


...


Results for sample size 9000:
K-L Divergence: 0.0361
Standard Deviation: 0.0958

Processing sample size: 10000


  0%|          | 0/1000000 [00:00<?, ?it/s]


Learned Structure (Edges) for 10000 samples:
[('EI_encoded', 'IR_encoded'), ('SP_encoded', 'EI_encoded'), ('SP_encoded', 'IR_encoded')]


  0%|          | 0/9 [00:00<?, ?it/s]


Results for sample size 10000:
K-L Divergence: 0.0889
Standard Deviation: 0.1316

All results have been saved to 'kl_std_results_mmhc.csv'.
   Sample_Size  K-L_Divergence  Standard_Deviation
0         1000        6.230031            0.281635
1         2000        4.067034            0.238933
2         3000        5.614728            0.247083
3         4000        0.031313            0.082406
4         5000        0.188786            0.148022
5         6000        1.034131            0.224367
6         7000        0.130362            0.121467
7         8000        0.121473            0.145836
8         9000        0.036146            0.095776
9        10000        0.088877            0.131563
